# [threading.Thread]启动与停止线程

`threading` 库可以在单独的线程中执行任何的在 `Python` 中可以调用的对象。你可以创建一个 `Thread` 对象并将你要执行的对象以 `target` 参数的形式提供给该对象

In [1]:
# Code to execute in an independent thread
import time
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

# Create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(10,))
t.start()

T-minus 10


当你创建好一个线程对象后，该对象并不会立即执行，除非你调用它的 `start()` 方法（当你调用 `start()` 方法时，它会调用你传递进来的函数，并把你传递进来的参数传递给该函数）。

`Python` 中的线程会在一个单独的系统级线程中执行（比如说一个 `POSIX` 线程或者一个 `Windows` 线程），这些线程将由操作系统来全权管理。

线程一旦启动，将独立执行直到目标函数返回。你可以查询一个线程对象的状态，看它是否还在执行：

```py
if t.is_alive():
    print('Still running')
else:
    print('Completed')

```

你也可以将一个线程加入到当前线程，并等待它终止：

```py
t.join()

```


后台线程无法等待，不过，这些线程会在主线程终止时自动销毁。 

除了如上所示的两个操作，并没有太多可以对线程做的事情。

你无法结束一个线程，无法给它发送信号，无法调整它的调度，也无法执行其他高级操作。

如果需要这些特性，你需要自己添加。

比如说，如果你需要终止线程，那么这个线程必须通过编程在某个特定点轮询来退出。

你可以像下边这样把线程放入一个类中：

In [23]:
%%file ThreadTest.py
import time
class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(5)

c = CountdownTask()

from threading import Thread
t = Thread(target=c.run, args=(10,))
t.start()
c.terminate() # Signal termination 信號終止
t.join()      # Wait for actual termination (if needed) 實際停止

Overwriting ThreadTest.py


In [22]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'ThreadTest.py')
cmd = "python " + f'"{DIR}"'
print(cmd,BASE_DIR,sep='\n')
#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程\ThreadTest.py"
D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程


如果线程执行一些像 `I/O` 这样的阻塞操作，那么通过轮询来终止线程将使得线程之间的协调变得非常棘手。

比如，如果一个线程一直阻塞在一个 `I/O` 操作上，它就永远无法返回，也就无法检查自己是否已经被结束了。

要正确处理这些问题，你需要利用超时循环来小心操作线程。 例子如下：

```py
class IOTask:
    def terminate(self):
        self._running = False

    def run(self, sock):
        # sock is a socket
        sock.settimeout(5)        # Set timeout period
        while self._running:
            # Perform a blocking I/O operation w/ timeout
            try:
                data = sock.recv(8192)
                break
            except socket.timeout:
                continue
            # Continued processing
            ...
        # Terminated
        return
```



## [multiprocessing]讨论

由于全局解释锁 `（GIL）` 的原因， `Python` 的线程被限制到同一时刻只允许一个线程执行这样一个执行模型。

所以， `Python` 的线程更适用于处理 `I/O` 和其他需要并发执行的阻塞操作（比如等待 `I/O` 、等待从数据库获取数据等等），而**不是需要多处理器并行的计算密集型任务**

你可以通过 `multiprocessing` 模块在一个单独的进程中执行你的代码：

In [18]:
import multiprocessing

class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(1)

c = CountdownTask()
p = multiprocessing.Process(
    target=c.run(n=5)
)
p.start()

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


# [threading.Event()]判断线程是否已经启动

线程的一个关键特性是每个线程都是独立运行且状态不可预测。

如果程序中的其他线程需要通过判断某个线程的状态来确定自己下一步的操作，这时线程同步问题就会变得非常棘手。

为了解决这些问题，我们需要使用 `threading` 库中的 `Event` 对象。 

`Event` 对象包含一个可由线程设置的信号标志，它允许线程等待某些事件的发生。

在初始情况下， `event` 对象中的信号标志被设置为假。

如果有线程等待一个 `event` 对象，而这个 `event` 对象的标志为假，那么这个线程将会被一直阻塞直至该标志为真。

一个线程如果将一个 `event` 对象的信号标志设置为真，它将唤醒所有等待这个 `event` 对象的线程。

如果一个线程等待一个已经被设置为真的 `event` 对象，那么它将忽略这个事件，继续执行。 

下边的代码展示了如何使用 `Event` 来协调线程的启动：

In [19]:
from threading import Thread, Event
import time

# Code to execute in an independent thread
def countdown(n, started_evt):
    print('countdown starting')
    # Event().set()
    started_evt.set()
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

# Create the event object that will be used to signal startup
started_evt = Event()

# Launch the thread and pass the startup event
print('Launching countdown')
t = Thread(target=countdown, args=(10,started_evt))
t.start()

# Wait for the thread to start
started_evt.wait()
print('countdown is running')

Launching countdown
countdown starting
T-minus 10


True

countdown is running


## [threading.Condition()]讨论

`event` 对象最好单次使用，就是说，你创建一个 `event` 对象，让某个线程等待这个对象，一旦这个对象被设置为真，你就应该丢弃它。

尽管可以通过 `clear()` 方法来重置 `event` 对象，但是很难确保安全地清理 `event` 对象并对它重新赋值。

很可能会发生错过事件、死锁或者其他问题（特别是，你无法保证重置 `event` 对象的代码会在线程再次等待这个 `event` 对象之前执行）。

如果一个线程需要不停地重复使用 `event` 对象，你最好使用 `Condition` 对象来代替。

下面的代码使用 `Condition` 对象实现了一个周期定时器，每当定时器超时的时候，其他线程都可以监测到：

In [25]:
%%file MultiThreadTest.py
import threading
import time

# ======= Timer =======

class PeriodicTimer:
    def __init__(self, interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()

    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True

        t.start()

    def run(self):
        '''
        Run the timer and notify waiting threads after each interval
        '''
        while True:
            time.sleep(self._interval)
            with self._cv:
                # 二進制 異位(XOR)
                self._flag ^= 1
                self._cv.notify_all()

    def wait_for_tick(self):
        '''
        Wait for the next tick of the timer
        '''
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()

# Example use of the timer
ptimer = PeriodicTimer(1)
ptimer.start()

# Two threads that synchronize on the timer
def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus', nticks)
        nticks -= 1

def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting', n)
        n += 1

threading.Thread(target=countdown, args=(10,)).start()
threading.Thread(target=countup, args=(5,)).start()

Overwriting MultiThreadTest.py


In [26]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'MultiThreadTest.py')
cmd = "python " + f'"{DIR}"'
print(cmd,BASE_DIR,sep='\n')
#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程\MultiThreadTest.py"
D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程


`event` 对象的一个重要特点是当它被设置为真时会唤醒所有等待它的线程。

如果你只想唤醒单个线程，最好是使用 **信号量** 或者 `Condition` 对象来替代。考虑一下这段使用信号量实现的代码：

In [42]:
import threading
# Worker thread
def worker(n, sema):
    # Wait to be signaled
    sema.acquire()

    # Do some work
    print('Working', n)

# Create some threads
sema = threading.Semaphore(0)
nworkers = 10
for n in range(nworkers):
    t = threading.Thread(target=worker, args=(n, sema,))
    t.start()

每次信号量被释放，只有一个线程会被唤醒并执行

In [43]:
sema.release()

sema.release()

Working 0
Working 1


# {情境及應用不瞭解}线程间通信

从一个线程向另一个线程发送数据最安全的方式可能就是使用 `queue` 库中的队列了。

创建一个被多个线程共享的 `Queue` 对象，这些线程通过使用 `put()` 和 `get()` 操作来向队列中添加或者删除元素。 例如：

In [44]:
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
    while True:
        # Produce some data
        ...
        out_q.put(data)

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()
        # Process the data
        ...

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))
t1.start()
t2.start()

`Queue` 对象已经包含了必要的锁，所以你可以通过它在多个线程间多安全地共享数据。 

当使用队列时，协调生产者和消费者的关闭问题可能会有一些麻烦。

一个通用的解决方法是在队列中放置一个特殊的值，当消费者读到这个值的时候，终止执行。例如：

```py
from queue import Queue
from threading import Thread

# Object that signals shutdown
_sentinel = object()

# A thread that produces data
def producer(out_q):
    while running:
        # Produce some data
        ...
        out_q.put(data)

    # Put the sentinel on the queue to indicate completion
    out_q.put(_sentinel)

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()

        # Check for termination
        if data is _sentinel:
            in_q.put(_sentinel)
            break

        # Process the data
        ...
```



本例中有一个特殊的地方：消费者在读到这个特殊值之后立即又把它放回到队列中，将之传递下去。

这样，所有监听这个队列的消费者线程就可以全部关闭了。 

尽管队列是最常见的线程间通信机制，但是仍然可以自己通过创建自己的数据结构并添加所需的锁和同步机制来实现线程间通信。

最常见的方法是使用 `Condition` 变量来包装你的数据结构。

下边这个例子演示了如何创建一个线程安全的优先级队列

```py
import heapq
import threading

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._count = 0
        self._cv = threading.Condition()
        
    def put(self, item, priority):
        with self._cv:
            heapq.heappush(self._queue, (-priority, self._count, item))
            self._count += 1
            self._cv.notify()

    def get(self):
        with self._cv:
            while len(self._queue) == 0:
                self._cv.wait()
            return heapq.heappop(self._queue)[-1]
```


使用队列来进行线程间通信是一个单向、不确定的过程。通常情况下，你没有办法知道接收数据的线程是什么时候接收到的数据并开始工作的。

不过队列对象提供一些基本完成的特性，比如下边这个例子中的 `task_done()` 和 `join()` ：

```py
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
    while running:
        # Produce some data
        ...
        out_q.put(data)

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()

        # Process the data
        ...
        # Indicate completion
        in_q.task_done()

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))
t1.start()
t2.start()

# Wait for all produced items to be consumed
q.join()
```


如果一个线程需要在一个 **“消费者”** 线程处理完特定的数据项时立即得到通知，你可以把要发送的数据和一个 `Event` 放到一起使用，这样 **“生产者”** 就可以通过这个 `Event` 对象来监测处理的过程了。示例如下：

```py
from queue import Queue
from threading import Thread, Event

# A thread that produces data
def producer(out_q):
    while running:
        # Produce some data
        ...
        # Make an (data, event) pair and hand it to the consumer
        evt = Event()
        out_q.put((data, evt))
        ...
        # Wait for the consumer to process the item
        evt.wait()

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data, evt = in_q.get()
        # Process the data
        ...
        # Indicate completion
        evt.set()
```


## [列隊判斷與線程]讨论

使用队列这种基于消息的通信机制可以被扩展到更大的应用范畴，比如，你可以把你的程序放入 **多个进程** 甚至是 **分布式系统** 而无需改变底层的队列结构。 

使用线程队列有一个要注意的问题是，向队列中 **添加数据项** 时并不会复制此数据项，线程间通信实际上是在线程间传递对象引用。

如果你担心对象的 **共享状态** ，那你最好只传递不可修改的数据结构（如：整型、字符串或者元组）或者一个对象的 **深拷贝** 。

```py
from queue import Queue
from threading import Thread
import copy

# A thread that produces data
def producer(out_q):
    while True:
        # Produce some data
        ...
        out_q.put(copy.deepcopy(data))

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()
        # Process the data
        ...
```


`Queue` 对象提供一些在当前上下文很有用的附加特性。

比如在创建 `Queue` 对象时提供可选的 `size` 参数来限制可以添加到队列中的元素数量。

对于 **“生产者”** 与 **“消费者”** 速度有差异的情况，为队列中的元素数量添加上限是有意义的。

比如，一个 **“生产者”** 产生项目的速度比 **“消费者”** **“消费”** 的速度快，那么使用固定大小的队列就可以在队列已满的时候阻塞队列，以免未预期的连锁效应扩散整个程序造成死锁或者程序运行失常。

在通信的线程之间进行“流量控制”是一个看起来容易实现起来困难的问题。

如果你发现自己曾经试图通过摆弄队列大小来解决一个问题，这也许就标志着你的程序可能存在脆弱设计或者固有的可伸缩问题。 **get()** 和 **put()** 方法都 **支持非阻塞方式和设定超时** ，例如：

```py
import queue
q = queue.Queue()

try:
    data = q.get(block=False)
except queue.Empty:
    ...

try:
    q.put(item, block=False)
except queue.Full:
    ...

try:
    data = q.get(timeout=5.0)
except queue.Empty:
    ...
```

这些操作都可以用来避免当执行某些特定队列操作时发生无限阻塞的情况，比如，一个非阻塞的 `put()` 方法和 **一个固定大小的队列** 一起使用，这样当队列已满时就可以执行不同的代码。

比如输出一条日志信息并丢弃。

```py
def producer(q):
    ...
    try:
        q.put(item, block=False)
    except queue.Full:
        log.warning('queued item %r discarded!', item)
```

如果你试图让消费者线程在执行像 `q.get()` 这样的操作时，超时自动终止以便检查终止标志，你应该使用 `q.get()` 的可选参数 `timeout` ，如下：

```py
_running = True

def consumer(q):
    while _running:
        try:
            item = q.get(timeout=5.0)
            # Process item
            ...
        except queue.Empty:
            pass
```

最后，有 `q.qsize()` ， `q.full()` ， `q.empty()` 等实用方法可以获取一个队列的当前大小和状态。但要注意，这些方法都不是线程安全的。

可能你对一个队列使用 `empty()` 判断出这个队列为空，但同时另外一个线程可能已经向这个队列中插入一个数据项。

所以，你最好不要在你的代码中使用这些方法。


# [threading; with]给关键部分加锁

要在多线程程序中安全使用可变对象，你需要使用 `threading` 库中的 `Lock` 对象，就像下边这个例子这样：

```py
import threading

class SharedCounter:
    '''
    A counter object that can be shared by multiple threads.
    '''
    def __init__(self, initial_value = 0):
        self._value = initial_value
        self._value_lock = threading.Lock()

    def incr(self,delta=1):
        '''
        Increment the counter with locking
        '''
        with self._value_lock:
             self._value += delta

    def decr(self,delta=1):
        '''
        Decrement the counter with locking
        '''
        with self._value_lock:
             self._value -= delta
```

`Lock` 对象和 `with` 语句块一起使用可以保证互斥执行，就是每次只有一个线程可以执行 `with` 语句包含的代码块。

`with` 语句会在这个代码块执行前自动获取锁，在执行结束后自动释放锁。


## [無明確應用，難以理解，未完]讨论

线程调度本质上是不确定的，因此，在多线程程序中错误地使用锁机制可能会导致随机数据损坏或者其他的异常行为，我们称之为竞争条件。

为了避免竞争条件，最好只在临界区（对临界资源进行操作的那部分代码）使用锁。

显式获取和释放锁是很常见的。下边是一个上一个例子的变种：


```py
import threading

class SharedCounter:
    '''
    A counter object that can be shared by multiple threads.
    '''
    def __init__(self, initial_value = 0):
        self._value = initial_value
        self._value_lock = threading.Lock()

    def incr(self,delta=1):
        '''
        Increment the counter with locking
        '''
        # 上 锁
        self._value_lock.acquire()
        self._value += delta
        # 释放锁
        self._value_lock.release()

    def decr(self,delta=1):
        '''
        Decrement the counter with locking
        '''
        # 上 锁
        self._value_lock.acquire()
        self._value -= delta
        # 释放锁
        self._value_lock.release()
```




相比于这种显式调用的方法， `with` 语句更加优雅，也更不容易出错，

特别是程序员可能会忘记调用 `release()` 方法或者程序在获得锁之后产生异常这两种情况（使用 `with` 语句可以保证在这两种情况下仍能正确释放锁）。 

为了避免出现死锁的情况，使用锁机制的程序应该设定为每个线程一次只允许获取一个锁

**未完**

# 防止死锁的加锁机制

在多线程程序中，死锁问题很大一部分是由于线程同时获取多个锁造成的。

举个例子：一个线程获取了第一个锁，然后在获取第二个锁的 时候发生阻塞，那么这个线程就可能阻塞其他线程的执行，从而导致整个程序假死。 

解决死锁问题的一种方案是为程序中的每一个锁分配一个唯一的`id`，然后只允许按照升序规则来使用多个锁，这个规则使用上下文管理器 是非常容易实现的，示例如下：

In [47]:
import threading
from contextlib import contextmanager

# Thread-local state to stored information on locks already acquired
_local = threading.local()

@contextmanager
def acquire(*locks):
    # Sort locks by object identifier
    locks = sorted(locks, key=lambda x: id(x))

    # Make sure lock order of previously acquired locks is not violated
    acquired = getattr(_local,'acquired',[])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock Order Violation')

    # Acquire all of the locks
    acquired.extend(locks)
    _local.acquired = acquired

    try:
        for lock in locks:
            lock.acquire()
        yield
    finally:
        # Release locks in reverse order of acquisition
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

# ======= 使用 ========
# 如何使用这个上下文管理器
# 不论是单个锁还是多个锁中都使用 acquire() 函数来申请锁        
import threading
x_lock = threading.Lock()
y_lock = threading.Lock()

def thread_1():
    while True:
        with acquire(x_lock, y_lock):
            print('Thread-1')

def thread_2():
    while True:
        with acquire(y_lock, x_lock):
            print('Thread-2')

t1 = threading.Thread(target=thread_1)
t1.daemon = True
t1.start()

t2 = threading.Thread(target=thread_2)
t2.daemon = True
t2.start()

Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1
Thread-2
Thread-1

如果你执行这段代码，你会发现它即使在不同的函数中以不同的顺序获取锁也没有发生死锁。 

其关键在于，在第一段代码中，我们对这些锁进行了排序。

通过排序，使得不管用户以什么样的顺序来请求锁，这些锁都会按照固定的顺序被获取。 

如果有多个 `acquire()` 操作被**嵌套**调用，可以通过线程本地存储 `（TLS）` 来检测潜在的死锁问题。 

假设你的代码是这样写的: 
```py
import threading
x_lock = threading.Lock()
y_lock = threading.Lock()

def thread_1():

    while True:
        with acquire(x_lock):
            with acquire(y_lock):
                print('Thread-1')

def thread_2():
    while True:
        with acquire(y_lock):
            with acquire(x_lock):
                print('Thread-2')

t1 = threading.Thread(target=thread_1)
t1.daemon = True
t1.start()

t2 = threading.Thread(target=thread_2)
t2.daemon = True
t2.start()
```

必定会有一个线程发生崩溃

```py
Exception in thread Thread-45:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-49-bb2fb04bd3bb>", line 15, in thread_2
    with acquire(x_lock):
  File "D:\anaconda\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "<ipython-input-47-57757de4db3d>", line 15, in acquire
    raise RuntimeError('Lock Order Violation')
RuntimeError: Lock Order Violation
```

发生崩溃的原因在于，每个线程都记录着自己已经获取到的锁。

`acquire()` 函数会检查之前已经获取的锁列表， 由于锁是按照升序排列获取的，所以函数会认为之前已获取的锁的 `id` 必定小于新申请到的锁，这时就会触发异常。

## 讨论

下面以一个关于线程死锁的经典问题：“哲学家就餐问题”，作为本节最后一个例子。

题目是这样的：

五位哲学家围坐在一张桌子前，每个人 面前有一碗饭和一只筷子。

在这里每个哲学家可以看做是一个独立的线程，而每只筷子可以看做是一个锁。

每个哲学家可以处在静坐、 思考、吃饭三种状态中的一个。

需要注意的是，每个哲学家吃饭是需要两只筷子的，这样问题就来了：如果每个哲学家都拿起自己左边的筷子， 那么他们五个都只能拿着一只筷子坐在那儿，直到饿死。

此时他们就进入了死锁状态。 

下面是一个简单的使用死锁避免机制解决“哲学家就餐问题”的实现：


```py
import threading

# The philosopher thread
def philosopher(left, right):
    while True:
        with acquire(left,right):
             print(threading.currentThread(), 'eating')

# The chopsticks (represented by locks)
NSTICKS = 5
chopsticks = [threading.Lock() for n in range(NSTICKS)]

# Create all of the philosophers
for n in range(NSTICKS):
    t = threading.Thread(
        target=philosopher,
        args=(chopsticks[n],chopsticks[(n+1) % NSTICKS])
    )
    t.start()
```
    
最后，要特别注意到，为了避免死锁，所有的加锁操作必须使用 `acquire()` 函数。如果代码中的某部分绕过 `acquire` 函数直接申请锁，那么整个死锁避免机制就不起作用了